The correlation classifier (implemented as a function here) takes a four-dimensional array of all subjects' data for all four conditions in an ROI (subjects x voxels x time x condition). Reps are not included in this analysis - it is possible to average over all three reps or just use rep 1. 


For each subject, the function correlates a voxel by time matrix from that subject in one condition with the VxT matrix of the other averaged subjects in all conditions. The correlation with the highest value is assigned as the prediction. If the predicted condition matches the actual condition, then the outcome is correct. This is performed for all four conditions and all subjects.


This implementation is a cross-subjects classifier, since that will be most useful in determining the utility of PCA and SRM. I expect that SRM with fewer features will perform the best in terms of improving cross-subject alignment. PCA will be used as a control for dimensionality reduction. I wouldn't be surprised if there's a ceiling effect with this analysis.


In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [67]:
def correlation_classifier(data):
  no_subj = data.shape[0]
  no_cond = data.shape[3]

  accuracy = np.zeros((no_cond,no_subj),dtype=bool)

  for s in range(no_subj):

    #print('subject %d'%s)

    # grab the data for this loop
    this_sub = data[s,:,:,:]
    avg_others = np.average(data[np.arange(no_subj)[np.arange(no_subj)!=s],:,:,:],axis=0)

    # for each condition
    for i in range(no_cond):

      # calculate the correlation value between this condition in this subject
      # and each condition in the average of other subjects
      corr_vals = np.zeros(no_cond)

      for j in range(no_cond):

        # grab the data for the conditions being tested
        this_flat = this_sub[:,:,i].flatten()
        avg_flat = avg_others[:,:,j].flatten()

        corr_vals[j] = np.corrcoef(this_flat,avg_flat)[0,1]

      #print('accuracy for cond %d'%i)
      #print(corr_vals)
      #print(i == np.argmax(corr_vals))

      # if the largest correlation is to the same condition, 
      # the prediction was correct
      accuracy[i,s] = i == np.argmax(corr_vals)
  
  #print()
  #print(accuracy)


  # return accuracy as a proportion correct across subjects
  return np.sum(accuracy, axis=1) / no_subj

In [ ]:
# create array of random values to test
# let's pretend this ROI has 100 voxels
data_rand = np.random.rand(4,100,148,4)
#correlation_classifier(data_rand)
acc = correlation_classifier(data_rand)
print(acc)

Now that it's working, let's load the real data from A1. (The following code was borrowed from `ryc_TRxTR-SRM_091321.ipynb`.)

In [71]:
roi = 'A1'
filepath = 'drive/MyDrive/fMRI_music_data/%s_by_subject_sorted_conds/'%roi
subjects = ['03','15','20','23']

In [72]:
# load in the data from all subjects
data = []
for subj in subjects: 
  data.append(np.load(filepath+'%s_sub-1%s.npy'%(roi,subj)))

# recast the list into an array and check the shape
data = np.asarray(data)
print(data.shape)

(4, 516, 1776)


Right now, this is all of the data from all of the reps. For this analysis, I will take only the data from rep 1.

In [74]:
n_sub = len(subjects)
n_voxels = data.shape[1]
n_TRs = 148
n_conds = 4
n_runs = 3

orig_data = np.copy(data)
orig_data = np.reshape(orig_data,(n_sub,n_voxels,n_TRs,n_conds,n_runs),order='F')
print(orig_data.shape)

(4, 516, 148, 4, 3)


In [75]:
rep1_data = orig_data[:,:,:,:,0]
print(rep1_data.shape)

(4, 516, 148, 4)


In [76]:
# run the rep 1 data through the classifier
true_acc = correlation_classifier(rep1_data)

subject 0
accuracy for cond 0
[0.02197606 0.01385221 0.02251754 0.01839836]
False
accuracy for cond 1
[0.00772357 0.0294541  0.02375911 0.01051152]
True
accuracy for cond 2
[0.04739856 0.01224783 0.02841029 0.02034555]
False
accuracy for cond 3
[0.01153964 0.00089649 0.00539758 0.02192166]
True
subject 1
accuracy for cond 0
[0.02766756 0.00578623 0.04525585 0.0109404 ]
False
accuracy for cond 1
[ 0.00807048  0.04345539  0.02055492 -0.00563804]
True
accuracy for cond 2
[0.03514422 0.01475202 0.01689835 0.00266933]
False
accuracy for cond 3
[ 0.01608528 -0.02597819 -0.00084698  0.00626816]
False
subject 2
accuracy for cond 0
[0.02654595 0.00860034 0.03373211 0.02190443]
False
accuracy for cond 1
[ 0.01562842  0.03467747  0.01207248 -0.00098585]
True
accuracy for cond 2
[ 0.0288729   0.03727881  0.02547191 -0.00449937]
False
accuracy for cond 3
[ 0.01328347 -0.00108772  0.02004571  0.02516735]
True
subject 3
accuracy for cond 0
[0.0315967  0.02207762 0.02080318 0.00114812]
True
accuracy f

In [78]:
print(true_acc)

[0.25 1.   0.25 0.75]


This means that for all subjects, the 8B matrix was best correlated with the 8B matrix of average other subjects. It is interesting that 1B is the next most accurate, and Intact and 2B are the worst. We can also see (from the previous cell's output) that the classifier was accurate for all conditions with s123, but not the other subjects. Looking at previously calculated ISC values (in the temporal ISC folder from the spring, `all_ROIs_AM/values.xlsx`), s123's ISC values aren't noticeably higher than the other subjects' values in A1.